## 머신러닝 실습

### 텍스트 마이닝

#### 개념

##### 텍스트 마이닝
- 비정형 텍스트데이터에서 패턴을 찾음. 의미가 있는 정보들을 추출하고 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석기법 이용
- 프로세스 - 텍스트전처리 > 특성 벡터화 > 머신러닝 모델 구축,학습,평가
- 텍스트 전처리 - 토큰화, 불용어 제거, 표제어 추출, 형태소 분석..

##### 특성 벡터화/ 추출
- 컴퓨터는 글을 읽을 수 없음 -> 단어기반으로 특성 추출, 숫자형 벡터값으로 표현
- BoW, Word2ve 라이브러리(모듈) 존재

##### LDA
- 문서에 잠재되어있는 토픽을 추론하는 확률 모델 알고리즘
- pyLDAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- rating.txt, ratings_train.txt, ratings_test.txt 다운로드
- id(리뷰번호),document(리뷰),label(감성분류클래스 0은 부정,1은 긍정 감성)

In [73]:
## 필요라이브러리 사용등록
import pandas as pd
import re

In [74]:
pd.__version__

'2.2.1'

In [75]:
## 버전 상세정보
pd.show_versions()


INSTALLED VERSIONS
------------------
commit                : bdc79c146c2e32f2cab629be240f01658cfb6cc2
python                : 3.12.2.final.0
python-bits           : 64
OS                    : Windows
OS-release            : 10
Version               : 10.0.19045
machine               : AMD64
processor             : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
byteorder             : little
LC_ALL                : None
LANG                  : None
LOCALE                : Korean_Korea.949

pandas                : 2.2.1
numpy                 : 1.26.4
pytz                  : 2024.1
dateutil              : 2.9.0.post0
setuptools            : 69.1.1
pip                   : 24.0
Cython                : None
pytest                : None
hypothesis            : None
sphinx                : None
blosc                 : None
feather               : None
xlsxwriter            : None
lxml.etree            : 5.1.0
html5lib              : None
pymysql               : None
psycopg2             

In [76]:
## 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('./data/ratings_train.txt', engine='python', sep='\t', encoding='utf-8')

In [77]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [78]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [79]:
dfNsmcTrain[dfNsmcTrain['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [80]:
## 결측치 확인
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'].notnull()]

In [81]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [82]:
dfNsmcTrain['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [83]:
## Regular Expressin(정규식)으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^ 가-힣|ㄱ-]+', '',x) )

In [84]:
dfNsmcTrain.tail()

,id,document,label
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [85]:
## 평가용 데이터 준비
dfNsmcTest = pd.read_csv('./data/ratings_test.txt', engine='python', sep='\t', encoding='utf-8')

In [86]:
dfNsmcTest.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [87]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [88]:
dfNsmcTest[dfNsmcTest['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [89]:
dfNsmcTest[dfNsmcTest['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [90]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'].notnull()]

In [91]:
dfNsmcTest['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [92]:
## Regular Expressin(정규식)으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^ 가-힣|ㄱ-]+', '',x) )

In [93]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' '데이터를 제거
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'] != ' ']

In [94]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149885 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149885 non-null  int64 
 1   document  149885 non-null  object
 2   label     149885 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [95]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터가 554개 존재
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'] != '']

In [96]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


##### 분석모델 구축

- 이전, 특징 벡터화 작업, 단어로 분리(토큰화) 후 TF-IDF 방식으로 벡터화
- 한글 형태소 분석은 konlpy 패키의 Okt 클래스 사용

##### KoNLPy 라이브러리 설치
- JDK 설치
- 시스템등록정보 JAVA_HOME 설정
    - 프로그램 실행 : sysdm.cpl
    - 시스템속성 > 고급탭 > 환경변수 버튼 클릭
    - 시스템 변수 > 새로 만들기 버튼 클릭
    - JAVA_HOME 과 경로 추가 등록

- JPype1 - 파이썬에서 JDK를 사용하게 해주는 프로그램

In [97]:
# Jpype1 설치
!pip install JPype1

In [98]:
# koNLPy 설치
!pip install koNLPy

In [99]:
import konlpy

In [100]:
konlpy.__version__

'0.6.0'

In [101]:
## 특성 벡터화 작업
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

In [102]:
okt = Okt()

In [103]:
def oktTokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [104]:
tfidf =  TfidfVectorizer(tokenizer=oktTokenizer, ngram_range=(1,2),min_df=3, max_df=0.9)

In [105]:
tfidf.fit(dfNsmcTrain['document'])
nsmc_train_tfidf = tfidf.transform(dfNsmcTrain['document'])

c:\DEV\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
with open('result.npy', 'wb') as f:
np.save(f, nsmc_train_tfisf)

IndentationError: expected an indented block after 'with' statement on line 1 (1832077803.py, line 2)

In [ ]:
nsmc_train_tfidf

<149885x113861 sparse matrix of type '<class 'numpy.float64'>'
	with 2672034 stored elements in Compressed Sparse Row format>

##### 감성 분류모델 구축

In [ ]:
## 로지스틱 회귀 라이브러리 등록
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(random_state=0)

In [ ]:
## 감성 이진분류 모델 훈련
model.fit(nsmc_train_tfidf,dfNsmcTrain['label'])

LogisticRegression(random_state=0)

In [ ]:
# 로지스틱 회귀 사용될 하이퍼파라미터 리스트
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [ ]:
## 최적 예측모델을 찾기위한 작업
# 추가라이브러리 사용등록
from sklearn.model_selection import GridSearchCV


In [ ]:
## 하이퍼파라미터 중 C(현재 1.0)인 값을 변경하면서 최적예측 모델 찾기
params = {'C' : [3.0,3.5,4.0,4.5,5.0]}
testModel = GridSearchCV(model,param_grid=params,cv=3,scoring='accuracy',verbose=1)

In [ ]:
testModel.fit(nsmc_train_tfidf,dfNsmcTrain['label'])

Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\DEV\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\DEV\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optim

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [3.0, 3.5, 4.0, 4.5, 5.0]}, scoring='accuracy',
             verbose=1)

In [ ]:
testModel.best_estimator_

LogisticRegression(C=4.5, random_state=0)

In [ ]:
# C=3.5로 해서 bestModel을 생성
bestModel = testModel.best_estimator_

##### 검증(평가)용 데이터 벡터화

In [ ]:
nsmc_test_tfidf = tfidf.transform(dfNsmcTest['document'])

In [ ]:
# 예측결과
y_predict = bestModel.predict(nsmc_test_tfidf)

In [ ]:
len(y_predict)

49997

In [ ]:
# 평가지표로 정확도 확인
from sklearn.metrics import accuracy_score

In [ ]:
# 베스트모델로 예측결과
accuracy_score(dfNsmcTest['label'],y_predict)

0.856831409884593

## 베스트모델로 감성예측

In [ ]:
sentence = input('리뷰입력 >> ')

In [ ]:
# 입력 문장에서 한글만 추출
st = re.compile(r'[가-힣|ㄱ-]').findall(sentence)
st

['웃', '자', '오', '늘', '은', '좋', '은', '날', '이', '될', '것', '같', '은', '예', '감']

In [ ]:
# 입력 문장에서 한글 외에 제거하고 다시 문장으로 할당
st = re.sub(r'[^가-힣|ㄱ-]+', ' ',sentence)
st

'웃자 오늘은 좋은 날이 될 것 같은 예감 '

In [ ]:
## 리스트로 형변환
listSt = [st]
listSt

['웃자 오늘은 좋은 날이 될 것 같은 예감 ']

In [ ]:
# 입력 텍스트의 벡터화
st_tfidf = tfidf.transform(lstSt)
print(st_tfidf)

NameError: name 'lstSt' is not defined

In [ ]:
# 0은 부정적인 감정, 1은 긍정적인 감정
bestModel.predict(st_tfidf)

In [ ]:
str_predict == 1